# Imports and configuration

In [1]:
import asyncio
import os
import pandas as pd
import time
from typing import Dict
import sys


from src.processing.constants import BASE_DIRECTORY
from src.preparation.models.obtained_text import ObtainedText
from src.preparation.models.descriptive_index import DescriptiveIndex
from src.preparation.models.connective_index import ConnectiveIndex
from src.preparation.models.lexical_diversity_index import LexicalDiversityIndex
from src.preparation.models.readability_index import ReadabilityIndex
from src.preparation.models.referential_cohesion_index import ReferentialCohesionIndex
from src.preparation.models.syntactic_complexity_index import SyntacticComplexityIndex
from src.preparation.models.syntactic_pattern_density_index import SyntacticPatternDensityIndex
from src.preparation.models.word_information_index import WordInformationIndex
from src.preparation.data_access.obtained_text_da import ObtainedTextDA
from src.processing.text_complexity_analyzer import TextComplexityAnalyzer

/home/hans/Documentos/Tesis_Chatbot/.env/lib/python3.8/site-packages/sqlalchemy/orm/util.py:104: SAWarning: The 'delete-orphan' cascade option requires 'delete'.
  util.warn(


# Calculate text complexity indices

In [2]:
tca = TextComplexityAnalyzer('es')
da = ObtainedTextDA()
obtained_texts = da.select_all()

for ot in obtained_texts:
    if ot.descriptive_index is not None and ot.word_information_index is not None and ot.syntactic_pattern_density_index is not None and ot.syntactic_complexity_index is not None and ot.connective_index is not None and ot.lexical_diversity_index is not None and ot.readability_index is not None and ot.referential_cohesion_index:
        print(f'{ot.filename} Ya ha sido procesado anteriormente.')
    else:
        try:
            start = time.time()
            descriptive_row = tca.calculate_descriptive_indices_for_one_text(ot.text)
            word_count = descriptive_row['DESWC']
            mean_words_per_sentence = descriptive_row['DESSL']
            mean_syllables_per_word = descriptive_row['DESWLsy']
            ot.descriptive_index = DescriptiveIndex(**descriptive_row)
            ot.word_information_index = WordInformationIndex(**tca.calculate_word_information_indices_for_one_text(ot.text, word_count=word_count))
            ot.syntactic_pattern_density_index = SyntacticPatternDensityIndex(**tca.calculate_syntactic_pattern_density_indices_for_one_text(ot.text, word_count=word_count))
            ot.syntactic_complexity_index = SyntacticComplexityIndex(**tca.calculate_syntactic_complexity_indices_for_one_text(ot.text))
            ot.connective_index = ConnectiveIndex(**tca.calculate_connective_indices_for_one_text(ot.text, word_count=word_count))
            ot.lexical_diversity_index = LexicalDiversityIndex(**tca.calculate_lexical_diversity_density_indices_for_one_text(ot.text))
            ot.readability_index = ReadabilityIndex(**tca.calculate_readability_indices_for_one_text(ot.text, mean_words_per_sentence=mean_words_per_sentence, mean_syllables_per_word=mean_syllables_per_word))
            ot.referential_cohesion_index = ReferentialCohesionIndex(**tca.calculate_referential_cohesion_indices_for_one_text(text=ot.text))
            end = time.time()
            da.update(ot) # Save the indices for the current record       
            print(f'Tiempo demorado para {ot.filename}: {end - start} segundos.')
        except Exception as e:
            print(f'{ot.filename} no pudo ser procesado debido a un error en el procesamiento.')
            print(str(e))
            continue

-1
8
-1
8
-1
8
-1
8
-1
8
-1
8
-1
8
-1
8
-1
8
-1
8
-1
8


Process Process-225:
Process Process-229:
Process Process-230:
Process Process-231:
Process Process-227:
Process Process-228:


KeyboardInterrupt: 

# Visualizing  the data obtained

In [ ]:
da = ObtainedTextDA()
obtained_texts = da.select_all_as_dataframe()
obtained_texts